In [1]:
# 필요한 라이브러리 및 모듈을 임포트

!pip install lxml

import numpy as np
import lxml
import os

from lxml import etree

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 61.3 MB/s eta 0:00:0000:0100:01


In [2]:
# 객체 클래스를 정의

CLASSES = ["car", "bus", "truck"]

In [3]:
# YOLOv8 형식으로 좌표를 변환하는 함수를 정의

def to_yolov8(y):
    width = y[2] - y[0]
    height = y[3] - y[1]

    if width < 0 or height < 0:
        print("ERROR: negative width or height ", width, height, y)
        raise AssertionError("Negative width or height")
    return (y[0] + (width/2)), (y[1] + (height/2)), width, height

In [4]:
# XML 파일에서 객체 검출(annotation) 정보를 로드하는 함수를 정의합니다.

def load_xml_annotations(f):
    tree = etree.parse(f)
    anns = []
    for dim in tree.xpath("image"):
        image_filename = dim.attrib["name"]
        width = int(dim.attrib["width"])
        height = int(dim.attrib["height"])

        boxes = []
        for box in dim.xpath("box"):
            label = CLASSES.index(box.attrib["label"])
            xtl, ytl = box.attrib["xtl"], box.attrib["ytl"]
            xbr, ybr = box.attrib["xbr"], box.attrib["ybr"]

            if 'occluded' in box.attrib:
                del box.attrib['occluded']
            if 'order' in box.attrib:
                del box.attrib['order']

            xc, yc, w, h = to_yolov8([float(xtl), float(ytl), float(xbr), float(ybr)])
            boxes.append([label, round(xc/width, 5), round(yc/height, 5), round(w/width, 5), round(h/height, 5)])

        if boxes:
            anns.append([image_filename[:-4] + ".txt", width, height, boxes])

    return anns

In [5]:
# YOLOv8 형식의 텍스트 파일로 변환하여 저장하는 함수를 정의

def write_yolov8_txt(folder, annotation):
    out_filename = os.path.join(folder, annotation[0])

    with open(out_filename, "w+") as f:
        for box in annotation[3]:
            f.write("{} {} {} {} {}\n".format(box[0], box[1], box[2], box[3], box[4]))

In [6]:
# 모든 XML 파일을 YOLOv8 형식의 텍스트 파일로 변환하는 함수를 정의

def convert_all_xml_files(input_dir, output_dir, specific_strings):
    for filename in os.listdir(input_dir):
        if filename.endswith('.xml') and any(s in filename for s in specific_strings):
            xml_file_path = os.path.join(input_dir, filename)
            folder_name = os.path.splitext(filename)[0]
            os.makedirs(os.path.join(output_dir, folder_name), exist_ok=True)

            # XML을 TXT로 변환
            anns = load_xml_annotations(xml_file_path)

            for ann in anns:
                write_yolov8_txt(os.path.join(output_dir, folder_name), ann)

In [8]:
# valid 데이터 txt변환

input_dir = '/home/jupyter/datasets/xml_val'
output_dir = '/home/jupyter/datasets/val/labels'
specific_strings = ['CH01', 'CH02', 'CH03', 'CH04']

convert_all_xml_files(input_dir, output_dir, specific_strings)


In [9]:
#train 데이터 txt변환

input_dir = '/home/jupyter/datasets/xml_train'
output_dir = '/home/jupyter/datasets/train/labels'
specific_strings = ['CH01', 'CH02', 'CH03', 'CH04']

convert_all_xml_files(input_dir, output_dir, specific_strings)
